# Projeto 4 - Classificação binária breast classificando e salvando classificador

## 1. Importando bibliotecas

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
import torch.nn as nn
torch.__version__

'2.5.0'

In [2]:
torch.set_default_device("mps")
device = torch.device("mps")

# torch.set_default_device("cpu")
# device = torch.device("cpu")

In [3]:
print(torch.backends.mps.is_available())  # Deve retornar True
print(torch.backends.mps.is_built())  # Deve retornar True

True
True


## 2. Importando Dados

In [4]:
np.random.seed(123)
torch.manual_seed(123)

In [5]:
previsores = pd.read_csv("data/entradas_breast.csv")
classe = pd.read_csv("data/saidas_breast.csv")

In [6]:
previsores = torch.tensor(np.array(previsores), dtype = torch.float)
classe = torch.tensor(np.array(classe), dtype = torch.float)

In [8]:
classe.T

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
         0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
         0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1.,
         1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
         0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
         1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0.,
         0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1.,
         0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0.,
         1., 1., 0., 0., 1.,

## 3. Transformação dos dados para tensores

In [10]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(previsores, classe), 
                                           batch_size = 10, 
                                           shuffle = True, 
                                           pin_memory=False,  # MPS não suporta pin_memory=True
                                           generator=torch.Generator(device=device))

## 4. Construção do modelo

In [11]:
class classificador_torch(nn.Module):
    def __init__(self):
        super().__init__()

        # 30 -> 16 -> 16 -> 1
        self.dense0 = nn.Linear(30, 8)
        torch.nn.init.normal_(self.dense0.weight, mean=0.0, std=0.05)

        self.dense1 = nn.Linear(8, 8)
        torch.nn.init.normal_(self.dense1.weight, mean=0.0, std=0.05)

        self.dense2 = nn.Linear(8, 1)
        torch.nn.init.normal_(self.dense2.weight, mean=0.0, std=0.05)
        
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
        self.output = nn.Sigmoid()

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dropout(X)
        
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dropout(X)
        
        X = self.dense2(X)
        X = self.activation(X)
        X = self.dropout(X)

        X = self.output(X)
        return X

In [12]:
classificador_torch = classificador_torch()

In [13]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(classificador_torch.parameters(), lr=0.001, weight_decay = 0.0001)

## 5. Treinamento do modelo

In [15]:
for epoch in range(100):
    running_loss = 0.

    for data in train_loader:
        inputs, labels = data #previsores, classes - tensor com quantidade batch_size
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad() #zerar o gradiente para o calculo

        # Forward
        outputs = classificador_torch(inputs) # classificador.forward(inputs) -> previsões
        #print(outputs)

        # Calculo do Erro
        loss = criterion(outputs, labels) #
        #print(loss)

        # Backpropagation
        loss.backward()
        
        optimizer.step() #Atualiza os pesos

        # atualiza o erro
        running_loss += loss.item()

    print(f"Epoch: {epoch}, loss: {running_loss/len(train_loader)}")
        

Epoch: 0, loss: 0.6468717549976549
Epoch: 1, loss: 0.6233659736942827
Epoch: 2, loss: 0.596858613323747
Epoch: 3, loss: 0.6089388926823934
Epoch: 4, loss: 0.5951461844276964
Epoch: 5, loss: 0.5985754022472783
Epoch: 6, loss: 0.5862698539307243
Epoch: 7, loss: 0.5850634120012584
Epoch: 8, loss: 0.5762824295905599
Epoch: 9, loss: 0.5908573431926861
Epoch: 10, loss: 0.5882776447555476
Epoch: 11, loss: 0.6066541164590601
Epoch: 12, loss: 0.5758819214084692
Epoch: 13, loss: 0.5794578748836852
Epoch: 14, loss: 0.5676391082897521
Epoch: 15, loss: 0.5939683961240869
Epoch: 16, loss: 0.5936395457962103
Epoch: 17, loss: 0.5754194646550898
Epoch: 18, loss: 0.5755977437161562
Epoch: 19, loss: 0.5757328313693666
Epoch: 20, loss: 0.5630255111476832
Epoch: 21, loss: 0.5956224537732309
Epoch: 22, loss: 0.5787233422722733
Epoch: 23, loss: 0.5801352910828173
Epoch: 24, loss: 0.5849914626594175
Epoch: 25, loss: 0.565313373218503
Epoch: 26, loss: 0.5635656510528765
Epoch: 27, loss: 0.5729621183453945
Epoc

## 6. Classificar somente um registro

In [16]:
novo_registro = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]], dtype = torch.float)

In [17]:
novo_registro

tensor([[1.5800e+01, 8.3400e+00, 1.1800e+02, 9.0000e+02, 1.0000e-01, 2.6000e-01,
         8.0000e-02, 1.3400e-01, 1.7800e-01, 2.0000e-01, 5.0000e-02, 1.0980e+03,
         8.7000e-01, 4.5000e+03, 1.4520e+02, 5.0000e-03, 4.0000e-02, 5.0000e-02,
         1.5000e-02, 3.0000e-02, 7.0000e-03, 2.3150e+01, 1.6640e+01, 1.7850e+02,
         2.0180e+03, 1.4000e-01, 1.8500e-01, 8.4000e-01, 1.5800e+02, 3.6300e-01]],
       device='mps:0')

In [19]:
classificador_torch.eval()

classificador_torch(
  (dense0): Linear(in_features=30, out_features=8, bias=True)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (dense2): Linear(in_features=8, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Sigmoid()
)

In [29]:
previsao = classificador_torch(novo_registro)

In [30]:
previsao

tensor([[1.]], device='mps:0', grad_fn=<SigmoidBackward0>)

In [31]:
previsao = previsao.detach()

In [32]:
previsao

tensor([[1.]], device='mps:0')

In [33]:
previsao = previsao.cpu()

In [34]:
previsao = previsao.numpy()
previsao

array([[1.]], dtype=float32)

In [36]:
previsao = (previsao > 0.5)
previsao

array([[ True]])

## 7. Salvar o modelo

In [37]:
classificador_torch.state_dict() 

OrderedDict([('dense0.weight',
              tensor([[ 7.4574e-02, -1.7704e-02, -2.2785e-03,  5.7946e-03, -1.6917e-02,
                        9.6920e-03, -1.9103e-02, -3.1056e-02, -3.1934e-03,  1.5530e-04,
                       -1.0100e-02, -2.0573e-02, -5.6476e-03,  1.8085e-04,  1.1633e-33,
                        1.5585e-08, -1.6096e-04, -9.8718e-34,  1.3048e-08,  3.1394e-11,
                        2.7221e-02, -3.1077e-02, -8.1115e-02, -6.0618e-02,  8.6933e-02,
                       -3.1037e-02, -1.7792e-02, -8.7776e-03, -8.0101e-02, -3.9559e-03],
                      [ 4.4589e-02,  2.1079e-02,  4.5174e-03, -3.9000e-02,  5.9652e-09,
                        4.0631e-03,  2.2769e-04, -2.4859e-05,  1.6553e-02,  9.5434e-08,
                       -4.6133e-03,  5.6078e-04, -1.4856e-02,  1.1423e-01, -2.3690e-19,
                       -2.6677e-18, -7.9293e-04,  5.5048e-19, -2.8171e-18,  1.6550e-19,
                       -5.7186e-03,  3.4836e-02, -4.0424e-02,  1.5331e-03,  2.1007e-02,


In [38]:
torch.save(classificador_torch.state_dict(), 'models/classificador.pth')